In [46]:
import pandas as pd
pd.set_option('display.max_columns', 300)

import numpy as np
import pickle
from pickle import load
from sklearn.linear_model import LinearRegression
import pandas as pd
import pickle
import numpy as np
import joblib
# pd.options.display.float_format = '{:,.0f}'.format 

## Step 1: Read in hold out data, scalers, and best model

In [55]:
#final_scaler = pickle.load(open('scaler.pickle','rb'))
final_model = pickle.load(open('model.pickle','rb'))
holdout = pd.read_csv('./data/kc_house_data_test_features.csv', index_col=0)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [56]:
holdout.columns

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [57]:
holdout[holdout['bedrooms'] == 33] = holdout[holdout['bedrooms'] == 3]
holdout['bathrooms'] = holdout.bathrooms.apply(lambda x: 1 if x < 1 else x)

In [62]:
df = holdout.copy()
df.head()
df_zip_dummies = pd.get_dummies(holdout['zipcode'],drop_first=True)
holdout2 = pd.concat([df, df_zip_dummies], axis=1)
holdout2.shape

(4323, 89)

In [63]:
transformed_holdout = holdout2.copy()
transformed_holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
transformed_holdout.columns

Index([           'id',          'date',      'bedrooms',     'bathrooms',
         'sqft_living',      'sqft_lot',        'floors',    'waterfront',
                'view',     'condition',         'grade',    'sqft_above',
       'sqft_basement',      'yr_built',  'yr_renovated',       'zipcode',
                 'lat',          'long', 'sqft_living15',    'sqft_lot15',
                 98002,           98003,           98004,           98005,
                 98006,           98007,           98008,           98010,
                 98011,           98014,           98019,           98022,
                 98023,           98024,           98027,           98028,
                 98029,           98030,           98031,           98032,
                 98033,           98034,           98038,           98039,
                 98040,           98042,           98045,           98052,
                 98053,           98055,           98056,           98058,
                 98059,  

## Step 3: Predict the holdout set

In [68]:
selected_columns= [     'bedrooms',     'bathrooms',   'sqft_living',        'floors',
          'waterfront',          'view',         'grade',    'sqft_above',
       'sqft_basement',  'yr_renovated',           'lat', 'sqft_living15',
               98002.0,         98003.0,         98004.0,         98006.0,
               98022.0,         98023.0,         98030.0,         98031.0,
               98033.0,         98038.0,         98039.0,         98040.0,
               98042.0,         98055.0,         98058.0,         98075.0,
               98092.0,         98102.0,         98105.0,         98106.0,
               98109.0,         98112.0,         98119.0,         98133.0,
               98168.0,         98178.0,         98198.0,         98199.0]

In [73]:
final_answers = final_model.predict(transformed_holdout[selected_columns])
final_answers = pd.DataFrame(final_answers)
final_answers.rename(columns={0:'price'})
final_answers.to_csv('housing_preds_ismael_araujo.csv')

## Step 4: Export your predictions